In [118]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '✔️',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()

Enpoint: whoami | Time: 0.5059723990125349 
Enpoint: people/475627 | Time: 0.48271679900062736 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.5350835980061674 
PROJECTS
Enpoint: projects | Time: 0.7616336270002648 
ROLES
Enpoint: roles | Time: 0.6055478719936218 
PEOPLE
Enpoint: people | Time: 0.5945824310038006 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.48869862699939404 
MILESTONES
Enpoint: milestones | Time: 0.4685551090078661 
ASSIGNMENTS
Enpoint: assignments | Time: 0.7180030739982612 
DONE! (5.2s)


In [127]:
names = list(fc.people.full_name)
names

['Oliver Strickson',
 'Angus Williams',
 'David Beavan',
 'Evelina Gabasova',
 'Giovanni Colavizza',
 'James Geddes',
 'James Hetherington',
 'Louise Bowler',
 "Martin O'Reilly",
 'May Yong',
 'Nick Barlow',
 'Radka Jersakova',
 'Timothy Hobson',
 'Tomas Lazauskas',
 'Camila Rangel Smith',
 'Sarah Gibson',
 'James Robinson',
 'Miguel Morin',
 'Eric Daub',
 'Jim Madge',
 'Amber Raza',
 'Kasra Hosseini',
 'Oscar Giles',
 'Joel Dearden',
 'Amaani Hoddoon',
 'Jack Roberts',
 'Oliver Forrest',
 'Gabriel Hanganu',
 'Ed Chalstrey',
 'Ashwini Venkatasubramaniam',
 'Roly Perera',
 'Mohammed Ali Al-Badri']

In [82]:
gid_mapping = {  # People without their full names on github
 'myyong': 'May Yong',
 'nbarlowATI': 'Nick Barlow',
 'thobson88': 'Timothy Hobson',
 'miguelmorin': 'Miguel Morin',
 'OscartGiles': 'Oscar Giles',
 'AshwiniKV': 'Ashwini Venkatasubramaniam',
}

Take an individuals average availability for the duration of a project and if >= project req green, if < req but > 0 Amber, if 0 red. You may have to use a Python date package like Pendulum for interpreting the dates so we can compare for a range.

In [137]:
peopledf = fc.people_totals.resample('MS').mean()  # numbers are already assigned

def get_person_availability(name, peopledf, start_date, end_date):
    try:
        person_id = fc.get_person_id(name)
    except:
        return None
    for index, row in peopledf.iterrows():
        if index.timestamp() < start_date or index.timestamp() > end_date:
            peopledf = peopledf.drop(index, axis=0)
    availability_range = peopledf[person_id]
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)
        
start_date = 1567296000.0
end_date = 1612137600.0

get_person_availability("Ed Chalstrey", peopledf, start_date, end_date)

0.39

In [70]:
resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
# resreqdf

In [129]:
fc.get_person_name(399979)
fc.get_person_id("Ed Chalstrey")
fc.projects.loc[1684539, "name"] # issue name
fc.projects.loc[1684539, "GitHub"] # issue num

'126'

In [42]:
issues = fc.projects["GitHub"].dropna().values
issues

array(['126', '122', '189', '153', '25', '158', '205', '270', '291', '306',
       '96', '76', '118', '174', '311', '286', '199', '31', '115', '64',
       '132', '46', '27', '106', '183', '50', '98', '190', '93', '243',
       '266', '295', '231', '240', '244', '175', '232', '238', '289',
       '230', '309', '308', '122', '317', '218', '234', '298', '238',
       '297', '229', '233', '302', '265', '237', '59', '176', '301', '149',
       '58', '134', '382', '217', '299', '239', '363', '360', '210', '359',
       '379', '381', '34', '332', '293', '28', '378', '377', '383'], dtype=object)

In [43]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

In [139]:
def tabulate_emojis(query_result, names, gid_mapping):
    data = [
        ["Person"],
        ["Rating"],
        ["Availability"]
    ]
    def add_to_data(laugh=False):
        for reaction in query_result['data']['repository']['issue']['reactionGroups']:
            if (reaction['content'] == 'LAUGH' and laugh) or (reaction['content'] != 'LAUGH' and not laugh):
                for edge in reaction['users']['edges']:
                    if edge['node']['name'] in names:
                        name = edge['node']['name']
                    elif edge['node']['login'] in gid_mapping:
                        name = gid_mapping[edge['node']['login']]
                    else:
                        name = edge['node']['login']
                        
                    data[0].append(name)

                    data[1].append(emojis[reaction['content']])
                    
                    data[2].append(get_person_availability(name, peopledf, start_date, end_date))
                
    add_to_data(laugh=True)
    add_to_data(laugh=False)

    display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
    ))


In [45]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

In [140]:
issues_with_resreq = []
for project_id in resreqdf:
    for resreq in resreqdf[project_id]:
        if resreq > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
            issue_num = fc.projects.loc[project_id, "GitHub"]
            if issue_num in issues:
                issues_with_resreq.append(issue_num)
                print(issue_num, fc.projects.loc[project_id, "name"],
                      "Start:", fc.projects.loc[project_id, "start_date"],
                      "End:", fc.projects.loc[project_id, "end_date"], 
                      round(resreq, 3), "FTE")
                
                modified_query = query.replace("X", str(issue_num))
                result = run_query(modified_query)  # Execute the query
                if query_result_contains_emojis(result):  # Currently only displaying those with at least 1 emoji
                    tabulate_emojis(result, names, gid_mapping)
                else:
                    print("NO EMOJIS FOUND")
            break

96 Living With Machines Start: 2018-06-01 End: 2022-03-31 1.0 FTE


Person,Giovanni Colavizza,Miguel Morin,Jack Roberts,Sarah Gibson,Kasra Hosseini,Nick Barlow,Camila Rangel Smith,Evelina Gabasova,Timothy Hobson,Radka Jersakova,James Robinson,Louise Bowler,Oscar Giles,Tomas Lazauskas,May Yong,Jim Madge
Rating,✅,✅,✅,✅,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌
Availability,1.0,1.0,0.5,0.44,0.67,0.31,0.19,0.61,0.61,0.11,0.44,0.53,0.11,0.42,0.25,0.33


311 AIDA-Lloyds Start: 2017-04-01 End: 2019-12-31 0.4 FTE


Person,Camila Rangel Smith,Evelina Gabasova,Sarah Gibson,Louise Bowler,Timothy Hobson,Giovanni Colavizza,James Robinson,Miguel Morin,Kasra Hosseini,Tomas Lazauskas,May Yong
Rating,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌
Availability,0.19,0.61,0.44,0.53,0.61,1.0,0.44,1.0,0.67,0.42,0.25


183 Probabilistic FEM Start: 2018-12-03 End: 2020-02-29 0.5 FTE


...

Person,Ed Chalstrey,Sarah Gibson,James Robinson,Louise Bowler,Jim Madge,Eric Daub,Jack Roberts,Nick Barlow,Evelina Gabasova,Oscar Giles,Radka Jersakova,Roly Perera,May Yong,Ashwini Venkatasubramaniam,Kasra Hosseini,Miguel Morin,Timothy Hobson
Rating,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌,❌,❌,❌
Availability,0.39,0.44,0.44,0.53,0.33,0.44,0.5,0.31,0.61,0.11,0.11,0.56,0.25,0.31,0.67,1.0,0.61


332 Scaleable Monte Carlo Start: 2020-10-01 End: 2021-09-30 1.0 FTE


Person,James Robinson,Nick Barlow,Louise Bowler,Jim Madge,Eric Daub,Ed Chalstrey,James Geddes,Oscar Giles,Miguel Morin,Tomas Lazauskas,Timothy Hobson,May Yong,Sarah Gibson,Roly Perera,Camila Rangel Smith,Evelina Gabasova,Kasra Hosseini,Ashwini Venkatasubramaniam,Radka Jersakova
Rating,✅,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌,❌,❌,❌,❌,❌
Availability,0.44,0.31,0.53,0.33,0.44,0.39,1.0,0.11,1.0,0.42,0.61,0.25,0.44,0.56,0.19,0.61,0.67,0.31,0.11


293 Wrattler 2019-2020 Start: 2019-04-01 End: 2020-03-31 1.0 FTE


Person,Sarah Gibson
Rating,✅
Availability,0.44
